In [2]:
from datasets import load_dataset

dataset1 = load_dataset("Xcvddax/Attack-techniques",split='train')
dataset2 = load_dataset("Xcvddax/Attack-mitigations",split='train')


c:\Users\User\Documents\code\Assistant_for_Attck\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
def format_func(datas):
    text = ""
    for i in range(len(datas)):
        for key in datas.column_names:
            text += f" {key}: {datas[key][i]} ,"
        text += "\n\n"
    return text

In [5]:
print(format_func(dataset1))

 type: x-mitre-tactic , spec_version: 2.1 , id: x-mitre-tactic--2558fd61-8c75-4730-94c4-11926db2a263 , created_by_ref: identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5 , created: 2018-10-17 00:14:20.652000+00:00 , modified: 2019-07-19 17:43:41.967000+00:00 , revoked: False , external_references: [{'external_id': 'TA0006', 'source_name': 'mitre-attack', 'url': 'https://attack.mitre.org/tactics/TA0006'}] , object_marking_refs: ['marking-definition--fa42a846-8d90-4e51-bc29-71d5b4802168'] , name: Credential Access , description: The adversary is trying to steal account names and passwords.

Credential Access consists of techniques for stealing credentials like account names and passwords. Techniques used to get credentials include keylogging or credential dumping. Using legitimate credentials can give adversaries access to systems, make them harder to detect, and provide the opportunity to create more accounts to help achieve their goals. , x_mitre_attack_spec_version: 2.1.0 , x_mitre_domain

In [5]:
print(dataset1.column_names)

['type', 'id', 'created_by_ref', 'created', 'modified', 'name', 'description', 'revoked', 'external_references', 'object_marking_refs', 'x_mitre_domains', 'x_mitre_modified_by_ref', 'x_mitre_version']


In [2]:
print(dataset1.features)

{'id': Value(dtype='string', id=None), 'attck_id': Value(dtype='string', id=None), 'attck_name': Value(dtype='string', id=None), 'description': Value(dtype='string', id=None), 'kill_chain_phases': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None), 'domains': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None), 'tactic_type': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None), 'embedding': Sequence(feature=Value(dtype='float32', id=None), length=-1, id=None)}


In [4]:
from dotenv import load_dotenv
import os 
load_dotenv()


from langchain_community.vectorstores.faiss import FAISS
from langchain_community.embeddings import OllamaEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_text(format_func(dataset1)+format_func(dataset2))

# graph
# embedding 
vectorstore = FAISS.from_texts(texts=splits, embedding=OllamaEmbeddings())
retriever = vectorstore.as_retriever()

In [ ]:
docs = retriever.invoke("")

In [1]:
from langchain import hub
prompt = hub.pull("rlm/rag-prompt")

In [ ]:
dir(prompt)

In [ ]:
example_messages = prompt.invoke(
    {"context": "filler context", "question": "filler question"}
).to_messages()
example_messages
print(example_messages[0].content)

In [6]:
from langchain_community.llms.ollama import Ollama

llm = Ollama()
llm.invoke("test")

'Hello! How can I assist you today? Is there something specific you would like to know or discuss?'

In [7]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

def format_docs(docs):
    return "\n\n".join( doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [ ]:
""" ref
T1651	Cloud Administration Command	Adversaries may abuse cloud management services to execute commands within virtual machines. Resources such as AWS Systems Manager, Azure RunCommand, and Runbooks allow users to remotely run scripts in virtual machines by leveraging installed virtual machine agents.
"""

"""
What are cloud management services, and how can they be abused by adversaries?
How do resources like AWS Systems Manager, Azure RunCommand, and Runbooks allow users to execute commands within virtual machines?
What are the potential risks of adversaries leveraging cloud management services to execute commands?
How can one identify and prevent adversaries from abusing cloud management services for command execution?
Do cloud providers offer security measures to prevent adversaries from abusing these management services?
"""

ans = rag_chain.invoke("What are cloud management services, and how can they be abused by adversaries?")

In [ ]:
import translators as ts
print(ts.translate_text(query_text=ans, translator='google', from_language= 'en', to_language='zh-TW'))